In [1]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
        "google/gemma-2-2b",
    )

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "/home/drdo/Caricatures/models/scan_gemma2-2b/checkpoint-95000",
    #quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    #attn_implementation='eager'
).to("cuda")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
ft_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/home/drdo/Caricatures/models/scan_gemma2-2b/checkpoint-95000")

In [5]:
#model.generation_config.max_length = 256
#model.to("cuda")

In [6]:
dataset = load_dataset("scan", "simple", trust_remote_code=True)

In [16]:
dataset['test'][1]

{'commands': 'run right twice after walk right twice',
 'actions': 'I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN'}

In [21]:
inputs = tokenizer(
    "run right twice after walk right twice", return_tensors="pt").to("cuda")
tokenizer.decode(ft_model.generate(**inputs, max_new_tokens=256)[0]).split('<bos>')[-1].split('<eos>')[0]

'I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK'

In [21]:
tokenizer.decode(model.generate(**inputs, max_new_tokens=256)[0].to("cpu"))

'<bos>run thrice and walk opposite left<bos>I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_RUN<eos>'

In [22]:
test_dataset = dataset['test'].select(range(100))

In [23]:
count = 0
bar = tqdm(range(len(test_dataset)))
for example in test_dataset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+" <sep>", return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256)[0].to("cpu")
    #output = model.generate(**inputs, num_beams=5, max_new_tokens=256)[0]
    output = tokenizer.decode(output).split('<bos>')[-1].split('<eos>')[0].strip()
    if output == label:
        count += 1
    bar.update(1)

print(count/len(test_dataset))

  0%|          | 0/100 [00:00<?, ?it/s]

0.05


In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968